In [2]:
import os

In [1]:
import torch
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config


def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt)#, map_location="cpu")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    model.cuda()
    model.eval()
    return model


def get_model():
    config = OmegaConf.load("configs/latent-diffusion/afhq-ldm-kl-16-mae-vae.yaml")  
    model = load_model_from_config(config, "logs/2024-04-27T19-42-30_afhq-ldm-kl-16-mae-vae/checkpoints/epoch=000149.ckpt")
    return model
def get_model_ijepa():
    config = OmegaConf.load("configs/latent-diffusion/afhq-ldm-kl-16-mae-vae.yaml")  
    model = load_model_from_config(config, "logs/2024-04-24T18-31-43_afhq-ldm-kl-16-jepa-vae/checkpoints/epoch=000149.ckpt")
    return model

In [2]:
from ldm.models.diffusion.ddim import DDIMSampler

model = get_model_ijepa()
sampler = DDIMSampler(model)


Loading model from logs/2024-04-24T18-31-43_afhq-ldm-kl-16-jepa-vae/checkpoints/epoch=000149.ckpt


/usr0/home/vishwaja/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 37.68 M params.
Keeping EMAs of 310.
Loaded quant_conv from checkpoint.
Working with z of shape (1, 16, 32, 32) = 16384 dimensions.
making attention of type 'vanilla' with 128 in_channels
Restored from logs/2024-04-26T19-44-33_autoencoder_kl_16x16x16_afhq_vae_mae/checkpoints/epoch=000099.ckpt
Training LatentDiffusion as an unconditional model.


In [7]:
import numpy as np 
from PIL import Image
from einops import rearrange
from torchvision.utils import make_grid
import os
from PIL import Image
import pytorch_lightning as pl

In [12]:
ddim_steps = 100
ddim_eta = 1.0
scale = 3.0   
all_samples = list()
folder_path = 'generated_samples/ijepa2/'
os.makedirs(folder_path, exist_ok=True)
i_t = 0
Nb = 2
batch_size = 16
from tqdm import tqdm
for j in tqdm(range(Nb)):
    pl.seed_everything(j)
    with torch.no_grad():
        with model.ema_scope():
            #   uc = model.get_learned_conditioning(
            #     {model.cond_stage_key: torch.tensor(10*[1000]).to(model.device)}
            #     )
            with model.ema_scope("Plotting"):
                    samples, z_denoise_row = model.sample_log(
                        cond=None,
                        batch_size=batch_size,
                        ddim=True,
                        ddim_steps=ddim_steps,
                        eta=ddim_eta,
                    )
                    # samples, z_denoise_row = self.sample(cond=c, batch_size=N, return_intermediates=True)
            x_samples = model.decode_first_stage(samples)
            
            # samples_ddim, _ = sampler.sample(S=ddim_steps,
            #                                         conditioning=None,
            #                                         batch_size=16,
            #                                         shape=[16, 64, 64],
            #                                         verbose=False,
            #                                         unconditional_guidance_scale=scale,
            #                                         unconditional_conditioning=None, 
            #                                         eta=ddim_eta)

            # x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples = torch.clamp((x_samples+1.0)/2.0, 
                                        min=0.0, max=1.0)
            all_samples.append(x_samples)

# Create a folder to save the images
    

# Iterate over the images and save each one as a file
        for i, image in enumerate(all_samples[0]):
            # Convert the tensor image to PIL Image
            img = 255*image.cpu().numpy().transpose(1, 2, 0)
            img = np.clip(img, 0, 255)
            pil_image = Image.fromarray(img.astype(np.uint8))

            # Save the image as a file
            file_path = os.path.join(folder_path, f'image_{i+i_t}.png')
            pil_image.save(file_path)

            
    i_t += batch_size

print('All images saved successfully.')

# grid = torch.stack(all_samples, 0)
# grid = rearrange(grid, 'n b c h w -> (n b) c h w')
# grid = make_grid(grid, nrow=n_samples_per_class)

# # to image
# grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
# Image.fromarray(grid.astype(np.uint8))



  0%|          | 0/2 [00:00<?, ?it/s]Global seed set to 0


Plotting: Switched to EMA weights
Data shape for DDIM sampling is (16, 16, 16, 16), eta 1.0
Running DDIM Sampling with 100 timesteps


 50%|█████     | 1/2 [00:01<00:01,  1.86s/it]Global seed set to 1


Plotting: Restored training weights
Plotting: Switched to EMA weights
Data shape for DDIM sampling is (16, 16, 16, 16), eta 1.0
Running DDIM Sampling with 100 timesteps


100%|██████████| 2/2 [00:03<00:00,  1.88s/it]

Plotting: Restored training weights
All images saved successfully.


In [4]:

import numpy as np
import os
from PIL import Image
from tqdm import tqdm
def load_images_to_array(directory):
    images = []
    filenames = os.listdir(directory)
    for filename in tqdm(filenames, total=len(filenames)):
        img_path = os.path.join(directory, filename)
        with Image.open(img_path) as img:
            if img.size[0] != 128:
                img = img.resize((128, 128))
            img_array = np.array(img)
            images.append(img_array)
    return np.array(images)

def save_images_as_npz(images, filename):
    np.savez_compressed(filename, images=images)
# Example Usage



In [ ]:
img_dir = 'logs/2024-04-24T18-31-43_afhq-ldm-kl-16-jepa-vae/samples/00022950/2024-04-28-16-55-45/img'
images_array = load_images_to_array(img_dir)
save_images_as_npz(images_array, 'images_afhq_ijepa.npz')

In [17]:
img_dir = 'logs/2024-04-27T19-42-30_afhq-ldm-kl-16-mae-vae/samples/00022950/2024-04-28-17-02-02/img'
images_array = load_images_to_array(img_dir)
save_images_as_npz(images_array, 'images_afhq_mae.npz')

In [20]:
img_dir = 'logs/2024-04-24T18-31-43_afhq-ldm-kl-16-jepa-vae/samples/00022950/2024-04-28-17-11-48/img'
images_array = load_images_to_array(img_dir)
save_images_as_npz(images_array, 'images_afhq_ijepa_5000_200.npz')

In [21]:
img_dir = 'logs/2024-04-27T19-42-30_afhq-ldm-kl-16-mae-vae/samples/00022950/2024-04-28-17-13-43/img'
images_array = load_images_to_array(img_dir)
save_images_as_npz(images_array, 'images_afhq_mae_5000_200.npz')

In [3]:
img_dir = 'logs/2024-04-24T18-31-43_afhq-ldm-kl-16-jepa-vae/samples_2024-04-28-22-51-17/img/epoch=000119'
epoch = os.path.basename(img_dir).split('=')[-1]
images_array = load_images_to_array(img_dir)
save_images_as_npz(images_array, f"images_npy/images_afhq_ijepa_{epoch}.npz")

NameError: name 'load_images_to_array' is not defined